In [ ]:
import xlwings as xw
import pandas as pd
from integration import RoseRocketIntegration
from backend import RoseRocketIntegrationBackend
import requests
from secretprod import secrets as pw
import pprint

In [ ]:
org='130'
rr = RoseRocketIntegration(org)
db = RoseRocketIntegrationBackend()
auth= rr.authorg(org)
headers = {
        'Accept': 'application/json',
        'Authorization': 'Bearer {}'.format(auth)


    }

In [ ]:
def getcarriercodebyid(carrierid,auth=auth):
    headers = {
        'Accept': 'application/json',
        'Authorization': 'Bearer {}'.format(auth)


    }
    apiurl = 'https://platform.sandbox01.roserocket.com/api/v1/partner_carriers/{}'.format(
                carrierid)
    try:
        resp=requests.get(apiurl,headers=headers).json()["partner_carrier"]["name"]
    except Exception as e:
        print(e)
        resp="NULL"
    return resp

In [ ]:
def getsalesorderidbyorderid(orderid,auth=auth):
    headers = {
        'Accept': 'application/json',
        'Authorization': 'Bearer {}'.format(auth)


    }
    apiurl="https://platform.sandbox01.roserocket.com/api/v1/orders/{}".format(orderid)
    try:
        salesorderno = requests.get(apiurl,headers=headers).json()["external_id"]
        return salesorderno
    except:
        return "NULL"

In [ ]:
def getlegs():
#     numpages=1
#     i=0
    legs=[]
#     while(i<numpages):
    apiurl="https://platform.sandbox01.roserocket.com/api/v1/manifests?status=bill-created"
    resp=requests.get(apiurl,headers=headers).json()
    
#     print(resp)
#         numpages=resp["total"]
#     for leg in resp["manifests"]:
#         print(leg)
#         legs.append(leg)
    return resp["manifests"]


In [ ]:
def writedata(data):
    import records
    
    db = records.Database("sqlite:///test.db")
    try:
        db.query("create table apimport(importkey varchar(50),whcode int, SCAC varchar(10), invoiceno varchar(15),  total5000 varchar(15),total6000 varchar(15), invoicedate date, duedate date, manifestid text)")
    except Exception as e:
        print(e)
        print("db already created?")
        
    query="""
    insert into apimport
        values(:importkey,:whcode,:SCAC, :invoiceno, :total5000, :total6000, :invoicedate, :duedate, :manifestid)
    """
    db.query(query,importkey=str(data["invoiceno"])+str(data["manifestid"])[:8],whcode=data["whcode"],SCAC=data["SCAC"],invoiceno=data['invoiceno'],total5000=data["total5000"],total6000=data["total6000"],invoicedate=data["invoicedate"],duedate=data["duedate"],manifestid=data["manifestid"])
    
    print("data logged! {}".format(data["invoiceno"]))
    

In [ ]:
def comparedata(data):
    #compare both lists and return which data is not in the database
    #for each row of that data not in the DB, use the writedata() method to write it
    actualdata=[]
    currentdata=RoseRocketIntegrationBackend()
    
    return actualdata

In [ ]:
data={
    "whcode":org,
    "SCAC":"",
    "invoiceno":"",
    
    
    "actualcost":"",
    
    "currency":"",
    "invoicedate":"",
    "duedate":"",
    "manifestid":""
    
    
    }
pddata=[]
legs=getlegs()
for leg in legs:
    #get salesorderno
#     data["salesorderno"]=getsalesorderidbyorderid(leg["order_id"])
#     print(leg)
    #get and set manifestid
    manifestid=leg["id"]
#     data["SCAC"]=getcarriercodebyid(leg["partner_carrier_id"])
    if(manifestid!=None):
        data["manifestid"]=manifestid
#         print(manifestid)
        #get manifest payments
#         apiurl="https://platform.sandbox01.roserocket.com/api/v1/manifests/{}/payment".format(manifestid)
#         payment=requests.get(apiurl,headers=headers).json()["manifest_payment"]
#         print(payment)
#         data["invoiceno"]=payment["id"]
#         print("invoice NO: {}".format(data["invoiceno"]))
#         if(int(payment["total_amount"])>0):
#             for item in payment["payment_items"]:
#                 data["billclass"]=item["bill_class"]
#                 data["billdesc"]=item["description"]
    #             data["unitcost"]=item["unit_price"]
    #             data["quantity"]=item["quantity"]
        apiurl="https://platform.sandbox01.roserocket.com/api/v1/bills?in_manifest_ids={}".format(manifestid)
        resp=requests.get(apiurl,headers=headers).json()
        if(len(resp["bills"])==0):
            continue
        else:
#             print(resp)
            billid=resp["bills"][0]["id"]
            apiurl="https://platform.sandbox01.roserocket.com/api/v1/bills/{}".format(billid)
            print(billid)
            resp=requests.get(apiurl,headers=headers).json()
#             print(resp)
#             data["actualcost"]=resp["total_amount"]
            data["invoiceno"]=resp["bill"]["reference_number"]

            data["invoicedate"]=resp["bill"]["bill_date"]
            data["duedate"]=resp["bill"]["due_date"]
            extra=0
            freightcost=0
            items=resp["items"]
            for item in items:
                if(item["bill_item_type"]["name"]!="Partner Freight"):
                    extra+=item["total_amount"]
                elif(item["bill_item_type"]["name"]=="Partner Freight"):
                    freightcost+=item["total_amount"]
                else:
                    print("item: {} is weird".format(item["bill_item_type"]["name"]))
            data["total5000"]=extra
            data["total6000"]=freightcost
#             print(resp["manifest_full_id"])
            try:
                if(resp["partner_carrier"]["standard_carrier_alpha_code"]!="null"):

                    data["SCAC"]=resp["partner_carrier"]["standard_carrier_alpha_code"]
                else:
                    
                    data["SCAC"]=str(resp["partner_carrier"]["name"])[:10]
            except:
                data["SCAC"]="NULL"
#                 data["carriername"]="CPU"
            print("end of bill logic")
#         print(data.copy())
        writedata(data.copy())
        pddata.append(data.copy())
    
print(pddata)
    
    
        